# Assignment 4

## Competition in Ready-Mix Concrete Manufacturing

![A Concrete Example](pics/readymixConcrete.jpg)

This assignment asks you to study competition in local markets of the Ready-Mix Concrete Manufacturing industry ([NAICS 327320](https://www.census.gov/econ/isp/sampler.php?naicscode=327320&naicslevel=6)). To this end, you can use the script below to estimate a simple version of [Abbring, Campbell, Tilly and Yang's (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf) dynamic model of oligopolistic entry, competition, and exit, using data from the U.S. Census Bureau. See e.g. the work of Chad Syverson and that of Allan Collard-Wexler for more information about and economic analyses of the Ready-Mix Concrete Manufacturing industry.

## Script for Estimating a Very Simple Industry Dynamics Model

This script is an adapted version of the script [example.m](example.m) in the Matlab package at [verysimple.abbring.org](http://verysimple.abbring.org). It calls various functions from that package, which we have included with this notebook. Please see [documentation.m.html](documentation.m.html) for detailed documentation of the original script and functions. Here, we only comment on parts of the script that are specific to this assignment.


### Numerical Settings 

In [2]:
Settings.tolInner = 1e-10;
Settings.tolOuter = 1e-6;
Settings.maxIter = 1000;

Settings.fdStep = 1e-7;

Settings.integrationLength = 32;
[Settings.integrationNodes, Settings.integrationWeights] = ...
    lgwt(Settings.integrationLength, 0, 1);
    
options = optimset( 'Algorithm', 'interior-point', 'Display', 'iter', ...
    'TolFun', Settings.tolOuter, 'TolX', Settings.tolOuter, ...
    'GradObj', 'off');

### Load and Process Data

The script requires the included text file [readymixConcrete.txt](readymixConcrete.txt), which contains annual observations of the numbers of Ready-Mix Concrete Manufacturing plants in 573 local markets (U.S. Micropolitan Statistical Areas) for the 2000–2009 period. It also contains the population of each local market in each year.

We have extracted these data from the Stata data file [markets.dta](markets.dta), which contains information on a range of six-digit NAICS industries (see [marketsDocumentation.txt](marketsDocumentation.txt)). For this, we have used the Stata `do` file [readymixConcrete.do](readymixConcrete.do), which can easily be adapted to extract data for other industries.

In [3]:
Data = importdata('readymixConcrete.txt',',', 1);
  
totalObs = size(Data.data,1);
Settings.tCheck = max(Data.data(:,2))+1-min(Data.data(:,2));
Settings.rCheck = totalObs/Settings.tCheck;

Data.N = reshape(Data.data(:,3),Settings.tCheck,Settings.rCheck); 
Settings.nCheck = max(max(Data.N));

Data.contC = Data.data(:,4)/10000;

Note that `Data.contC` measures population in tens of thousands of people. We need to discretize the population data on a finite grid. By changing the next three lines of the script, you can control the grid's coarseness and boundaries.  

In [4]:
Settings.cCheck = 1000;
Settings.lowBndC = 0.9*min(Data.contC);
Settings.uppBndC = 1.1*max(Data.contC);

Settings.logGrid = ...
linspace(log(Settings.lowBndC), log(Settings.uppBndC), Settings.cCheck);
Settings.d = Settings.logGrid(2) - Settings.logGrid(1);

temp = zeros(totalObs,1);
for i = 1:totalObs
    test = (log(Data.contC(i))-Settings.logGrid).^2;
    temp(i) = find(min(test)-test>=0,1);
end
Data.C = reshape(temp,Settings.tCheck,Settings.rCheck);

### Estimation

We compute maximum likelihood estimates using the three-step nested fixed-point procedure outlined in [Abbring et al. (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf). 

#### Step 1
The first step estimates the parameters of the population process directly from the population data.

In [5]:
from = Data.C(1:Settings.tCheck - 1, 1:Settings.rCheck);
to = Data.C(2:Settings.tCheck, 1:Settings.rCheck);

logTransitions = log(Data.C([from(:), to(:)]));
innovLogC = logTransitions(:, 2) - logTransitions(:, 1);
startValues.step1 = [mean(innovLogC)  std(innovLogC)];

objFunStep1 = @(estimates) likelihoodStep1(Data, Settings, estimates); 

tic;
[Estimates.step1, llh.step1, exitFlag.step1] = fmincon(objFunStep1, ...
    startValues.step1, [], [], [], [], [-inf, 0], [], [], options);
computingTime.step1 = toc;

                                            First-order      Norm of
 Iter F-count            f(x)  Feasibility   optimality         step
    0       3    2.705062e+04    0.000e+00    1.131e+03
Objective function returned Inf; trying a new point...
    1      11    2.217446e+04    0.000e+00    2.659e+04    1.546e-01
Objective function returned Inf; trying a new point...
    2      25    2.168956e+04    0.000e+00    1.208e+03    3.597e-02
Objective function returned Inf; trying a new point...
    3      29    1.876015e+04    0.000e+00    5.414e+04    4.346e-02
Objective function returned Inf; trying a new point...
    4      33    1.742903e+04    0.000e+00    2.273e+05    2.034e-02
Objective function returned Inf; trying a new point...
    5      38    1.609916e+04    0.000e+00    2.682e+05    8.026e-03
Objective function returned Inf; trying a new point...
    6      43    1.434513e+04    0.000e+00    1.601e+05    1.701e-02
    7      47    1.421917e+04    0.000e+00    2.700e+04    3.9

#### Step 2
The second step estimates the remaining parameters using a nested fixed-point maximum likelihood procedure, fixing the parameters of the population process to their first step estimates.

By changing the next five lines of the script, you can control the model's specification (note that this Matlab code, unlike the R/C++ code that was used to produce the simulation and empirical results in [Abbring et al. (2017)](https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf), does not allow for market heterogeneity or economic barriers to entry). 

In [6]:
Param.rho = 1/1.05; 
Settings.estimates2k = @(x) [x(1:3) x(4)*ones(1,Settings.nCheck-3)];
Settings.estimates2phi = @(x) x(5);
Settings.estimates2omega = @(x) x(6);
startValues.step2 = [0.4 0.4 0.4 0.4 25 1.6];

Param = markov(Param, Settings, Estimates.step1(1), Estimates.step1(2));

objFunStep2 = @(estimates) likelihoodStep2(Data, Settings, Param, estimates);

lb = zeros(size(startValues.step2));

tic;
[Estimates.step2,llh.step2,exitFlag.step2] = fmincon(objFunStep2, ...
    startValues.step2, [], [], [], [], lb, [], [], options);
computingTime.step2 = toc;

                                            First-order      Norm of
 Iter F-count            f(x)  Feasibility   optimality         step
    0       7    4.632981e+03    0.000e+00    1.561e+03
Objective function returned Inf; trying a new point...
    1      20    4.428971e+03    0.000e+00    2.417e+03    4.064e-01
    2      29    4.417286e+03    0.000e+00    2.180e+03    8.947e-02
Objective function returned Inf; trying a new point...
    3      40    4.321329e+03    0.000e+00    3.788e+03    2.779e-01
Objective function returned Inf; trying a new point...
    4      49    4.311846e+03    0.000e+00    1.113e+03    1.011e-01
Objective function returned Inf; trying a new point...
    5      58    4.244163e+03    0.000e+00    1.652e+03    1.372e-01
    6      67    4.205712e+03    0.000e+00    2.230e+03    2.785e-01
Objective function returned Inf; trying a new point...
    7      77    4.202177e+03    0.000e+00    2.157e+03    6.133e-01
    8      84    4.166436e+03    0.000e+00    4.

#### Step 3
The last step maximizes the full likelihood, starting from the previous steps' estimates, to produce efficient estimates and correct standard errors.  

In [7]:
startValuesStep3 = [Estimates.step2, Estimates.step1];

objFunStep3 = @(estimates) likelihoodStep3(Data, Settings, Param, estimates);

lb = zeros(size(startValuesStep3));
lb(length(startValuesStep3) - 1) = -inf;

tic;
[Estimates.step3, llh.step3, exitFlag.step3] = fmincon(objFunStep3,...
    startValuesStep3, [], [], [], [], lb, [], [], options);
computingTime.step3 = toc;

[~,Estimates.se] = likelihoodStep3(Data, Settings, Param, Estimates.step3);

                                            First-order      Norm of
 Iter F-count            f(x)  Feasibility   optimality         step
    0       9    1.816850e+04    0.000e+00    3.052e+01
Objective function returned Inf; trying a new point...

Local minimum possible. Constraints satisfied.

fmincon stopped because the size of the current step is less than
the selected value of the step size tolerance and constraints are 
satisfied to within the default value of the constraint tolerance.





In [8]:
Estimates
computingTime


Estimates = 

  struct with fields:

    step1: [0.0034 0.0112]
    step2: [0.3820 0.2558 0.2179 0.1692 27.7170 1.5966]
    step3: [0.3820 0.2558 0.2179 0.1692 27.7170 1.5966 0.0034 0.0112]
       se: [0.0149 0.0150 0.0155 0.0138 3.9164 0.0549 1.5275e-04 6.9314e-05]


computingTime = 

  struct with fields:

    step1: 1.1364
    step2: 469.8409
    step3: 48.1190



## Questions
### 1. Does it makes sense to treat Micropolitan Statistical Areas as distinct local markets of the Ready-Mix Concrete Manufacturing industry?

### 2. Interpret and discuss the estimation results.

## References

- Abbring, Jaap, Jeffrey Campbell, Jan Tilly and Nan Yang (2017; Revised April 2018). Very simple Markov-perfect industry dynamics: Empirics. Discussion Paper 2017-021. CentER, Tilburg. https://edubb.uvt.nl/bbcswebdav/pid-1678085-dt-content-rid-5981553_1/courses/325134-2017-2018/acty1b.pdf